In [1]:
# coding=utf-8
import pandas as pd
import numpy as np
from scipy.misc import comb

## (b) Select a 2-class dataset with discrete nD features. This dataset needs to be derived from text documents.


In [2]:
def rf(fn):
    data = pd.read_csv(fn, sep=',', header=None, skipinitialspace=True,)
    # print 'og data size',data.shape
    # print data.head()
    x = data.loc[:,1:]
    p = x.sum(axis = 1)
    data[45] = p
    # mdata = data.add(p,axis = 0)
    return data


# K fold
def k_fold(size, k, shuffle = False):
    index = np.arange(0,size)
    if shuffle:
        np.random.shuffle(index)
    index = np.reshape(index, (k,size/k))
    test = []
    train = []
    for i in range(0,k):
        test.append(index[i])
        temp = np.delete(index,i,0)
        train.append(temp.flatten())
    return test, train

def getParam(train_data):
    xnum = train_data.shape[0]
    fnum = train_data.shape[1]-2
    # p = train_data[fnum+1]
    alfaj = np.zeros((2,fnum))
    alfa = np.zeros(2)
    # p = np.zeros((2,xnum))
    sub = pd.DataFrame
    subc = [sub,sub]
    for i in range(0,2):
        subc[i] = train_data[train_data[0] == i]
        x_train = subc[i].loc[:,1:fnum]
        ps = subc[i][fnum+1]
        p = ps.sum()
        alfa[i] = len(x_train)
        alfaj[i] = x_train.sum(axis = 0)
        alfaj[i] = (alfaj[i]+1)/(alfa[i]*p+2)
    # subc0 = train_data[train_data[0] == 0]
    # subc1 = train_data[train_data[0] == 1]
    # alfa[0] = len(subc0)
    # alfa[1] = len(subc1)
    # alfaj[0] = subc0.sum(axis = 0) # column sum
    # alfaj[1] = subc1.sum(axis = 0)
    # p[0] = subc0.sum(axis = 1) # row sum
    # p[1] = subc1.sum(axis = 1)
    # for i in range(0,2):
    #
    #     alfaj[i] = (alfaj[i]+1)/(alfa[i]*p[i]+2)
    # print 'alfa shape',alfa.shape
    # print 'alfaj shape', alfaj.shape
    return alfa/xnum, alfaj


## (c) Estimate the model parameters and compute a discriminant function based on the distribution in each class. Make sure to use the Naive Bayes assumption.

In [3]:
def calcm(x,xa,xaj,pi): # x is an example, xa is the alfa of x's class, xaj is also alfaj of that class. so xa is 1*1, xaj is 1*n
    g = 0.0
    l = len(x)
    for i in range(0,l):
        g += (np.log(comb(pi,x[i+1])) + x[i+1]*np.log(xaj[i]) + (pi-x[i+1])*np.log(1-xaj[i]))
    return g+np.log(xa)


def classification(test_data, a, aj):
    xnum = test_data.shape[0]
    fnum = test_data.shape[1]-2
    x_test = test_data.loc[:,1:fnum]
    p = test_data[fnum+1].tolist()
    # y_test = test_data[0]
    prelist = []
    for i in range(0, xnum):
        ex = x_test.iloc[i]
        # print i
        c1 =  calcm(ex,a[0],aj[0],p[i])
        c2 =  calcm(ex,a[1],aj[1],p[i])
        if c1<c2:
            prelist.append(2)
        else:
            prelist.append(1)
    return prelist

def cMatrix(test_data,prelist):
    y_test= test_data[0].tolist()
    cm = np.zeros((2,2))
    print len(y_test)
    for i in range(0,len(y_test)):
        rl = y_test[i]
        pre = prelist[i]
        cm[rl-1][pre-1] += 1
    return cm

# Compute the confusion matrix, precision, recall,F-measure, and accuracy
def evalpf(cm):
    tp = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tn = cm[1][1]
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    Fm = 2*precision*recall/(precision+recall)
    return (precision,recall,Fm,accuracy)

if __name__ == "__main__":

    data_train = rf('SPECTF.train')
    data_test = rf('SPECTF.test')
    alfa, alfaj = getParam(data_train)  #(alfa/xnum, alfaj)
    print 'Alpha:'
    print alfa
    print 'Alphaj:'
    print alfaj

    y_predict = classification(data_test, alfa, alfaj)
    cm = cMatrix(data_test,y_predict)
    print 'confusion matrix is '
    print cm
    result = evalpf(cm)
    print 'result of precision, recall,F-measure, and accuracy are'
    print result

Alpha:
[ 0.5  0.5]
Alphaj:
[[ 0.00056361  0.00056942  0.00059016  0.00060676  0.00055863  0.00057585
   0.0005997   0.00060406  0.00053436  0.00054556  0.00058684  0.00059763
   0.00057834  0.00059327  0.00054639  0.00053975  0.00057211  0.00057149
   0.00056029  0.00055925  0.00062045  0.00062895  0.00058518  0.00059908
   0.00053892  0.00053851  0.0005327   0.00054245  0.00055842  0.00057108
   0.00060779  0.00062418  0.00052523  0.0005354   0.00054494  0.00054992
   0.00054577  0.00055386  0.00060053  0.0006051   0.00058166  0.0005773
   0.00049827  0.00048997]
 [ 0.00060987  0.0005844   0.00059758  0.00060285  0.00058791  0.00058001
   0.00063184  0.00061514  0.00056551  0.0005464   0.00060724  0.00060856
   0.00059692  0.0005945   0.00055277  0.000524    0.00059801  0.0005833
   0.00057825  0.00055475  0.00064765  0.00063096  0.00058989  0.00058484
   0.00049414  0.0004814   0.0005396   0.00051412  0.00054333  0.00052444
   0.00062327  0.00063205  0.00054113  0.00055299  0.0005688